In [7]:
import GPy
import string
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
#import cPickle as pickle
from GPy.util import pca
from matplotlib import pyplot as plt
import torch
from sklearn import svm
from sklearn.utils import resample
%matplotlib inline

In [8]:
def get_data(data_type,file_num):

    if(data_type=='train'):
        data=pd.read_csv(f"Dataset/IHDP_a/ihdp_npci_train_{file_num}.csv")
        #data=pd.read_csv(f"Data/ihdp_npci_train_{file_num}.csv",index_col=False)
    else:
        data = pd.read_csv(f"Dataset/IHDP_a/ihdp_npci_test_{file_num}.csv")
        #data = pd.read_csv(f"Data/ihdp_npci_test_{file_num}.csv",index_col=False)

    x_data=pd.concat([data.iloc[:,0], data.iloc[:, 1:30]], axis = 1)
 
   
    y_data=data.iloc[:, 1]
    
    return x_data,y_data

def cal_pehe(y_1,y_0,test_x):
    #pehe...
    cate_pred=y_1-y_0
    #cate_pred=y1-y0
    cate_true=test_x.iloc[:,4]-test_x.iloc[:,3] #Hill's noiseless true values
    cate_err=np.mean( np.square( ( (cate_true) - (cate_pred) ) ) )
    return np.sqrt(cate_err)

In [9]:
def train_GPLVM(input_dim,Yn,kr):
    input_dim = input_dim
    # define covariance function to be used
    if(kr=='RBF'):
        kernel = GPy.kern.RBF(input_dim, ARD=True)
    else:
        kernel = GPy.kern.Linear(input_dim, ARD=True)
        
    # setup the GPLVM model
    m = GPy.models.GPLVM(Yn, input_dim=input_dim, kernel=kernel)
    # optimize for 1000 iterations
    m.optimize(messages=1, max_iters=1000) 
    return m

In [10]:
def upsample(x_data, y_data):
    x_train_sr=x_data[x_data['treatment']==1]
    y_train_sr=y_data[x_data['treatment']==1]
    x_train_tr=x_data[x_data['treatment']==0]
    y_train_tr=y_data[x_data['treatment']==0]
    
    #x_sr_upsample = resample(x_train_sr,replace=True,n_samples=len(x_train_tr),random_state=42)
    
    #y_sr_upsample = resample(y_train_sr,replace=True,n_samples=len(y_train_tr),random_state=42)
    
    return x_train_sr,y_train_sr,x_train_tr,y_train_tr
    

def SA_2(sr,tr,lat_dim):
    pca_src_ = PCA(lat_dim)
    pca_tgt_ = PCA(lat_dim)
    pca_src_.fit(sr)
    pca_tgt_.fit(tr)
    Xs=pca_src_.components_.transpose()
    Xt=pca_tgt_.components_.transpose()
    
    sr_aligned=sr@(Xs @ Xs.transpose()@Xt);
    tr_projected = tr@Xt;
    
    
    return sr_aligned, tr_projected,pca_src_,pca_tgt_

    
def SA(sr,tr,lat_dim,kr):
    Ysr=sr.transpose()
    Ytr=tr.transpose()
    msr=train_GPLVM(lat_dim,Ysr,kr)
    mtr=train_GPLVM(lat_dim,Ytr,kr)
    emd_sr=np.array(msr.X)
    emd_tr=np.array(mtr.X)
    sr_aligned=sr@(emd_sr @ emd_sr.transpose()@emd_tr);
    tr_projected = tr@emd_tr;
    
    return sr_aligned,tr_projected,msr,mtr
    
    #color=np.where(labels==1, 'Red','Green')
    #ar=np.array(m.X)

In [11]:
#fil_num=50
input_dim=10
kr='Linear' #RBF
pehe_error=[]
reg_0 = svm.SVR()
reg_1=svm.SVR()

for i in range(1,101):
    x_data,y_data=get_data('train',i)
    test_x,test_y=get_data('test',i)
    x_sr,y_sr,x_tr,y_tr=upsample(x_data, y_data)
    
    #x_sr_tran=x_sr.iloc[:,5:30].to_numpy().transpose()
    #x_tr_tran=x_tr.iloc[:,5:30].to_numpy().transpose()
    #sr_aligned,emd_tr,msr,mtr=SA(x_sr_tran,x_tr_tran,input_dim,kr)
    sr,tr,pca_sr,pca_tr=SA_2(x_sr.iloc[:,5:30],x_tr.iloc[:,5:30],input_dim)
    
    # Test data transformation
    test_sr=test_x[test_x['treatment']==1]
    #test_sr_y=test_y[test_x['treatment']==1]
    test_tr=test_x[test_x['treatment']==0]
    #test_tr_y=test_y[test_x['treatment']==0]
    #test_tr=test_tr.iloc[0:test_sr.shape[0],:]
    
    sr_test=test_sr.iloc[:,5:30].to_numpy()
    tr_test=test_tr.iloc[:,5:30].to_numpy()
    
    Xs_test=pca_sr.components_.transpose()
    Xt_test=pca_tr.components_.transpose()
    
    sr_aligned_test=sr_test@(Xs_test @ Xs_test.transpose()@Xt_test);
    tr_projected_test= tr_test@Xt_test;
    
    
    test_complt=pd.concat([test_sr.iloc[:,0:5],test_tr.iloc[:,0:5]], axis=0)
    test_transformed_complt=np.concatenate((sr_aligned_test,tr_projected_test), axis=0)
    
    
    reg_1.fit(sr, y_sr)
    y_1=reg_1.predict(test_transformed_complt)
    reg_0.fit(tr, y_tr)
    y_0=reg_0.predict(test_transformed_complt)
    pehe=cal_pehe(y_1,y_0,test_complt)
    pehe_error.append(pehe)

In [12]:
np.mean(pehe_error)

1.3106601302127379

In [47]:
#t_sr_data=msr.infer_newX(test_sr.iloc[:,5:30].to_numpy())
#emd_sr_test=np.array(t_sr_data[0])
#t_tr_data=mtr.infer_newX(test_tr.iloc[:,5:30].to_numpy())
#emd_tr_test=np.array(t_tr_data[0])

#sr_aligned_test=(emd_sr_test @ emd_sr_test.transpose()@emd_tr_test);
#test_complt=pd.concat([test_sr.iloc[:,0:5],test_tr.iloc[:,0:5]], axis=0)
#test_embd_complt=np.concatenate((sr_aligned_test,emd_tr_test), axis=0)

#reg_1.fit(sr_aligned, y_sr)
#y_1=reg_1.predict(test_embd_complt)
#reg_0.fit(emd_tr, y_tr)
#y_0=reg_0.predict(test_embd_complt)
#pehe=cal_pehe(y_1,y_0,test_complt)
#pehe_error.append(pehe)

In [48]:
"""
# Principal Component Analysis
from numpy import array
from sklearn.decomposition import PCA
# define a matrix
A = array([[1, 2,7], [3, 4,8], [5, 6,9]])
print(A)
# create the PCA instance
pca = PCA(2)
# fit on data
pca.fit(A)
# access values and vectors
print(pca.components_)
print(pca.explained_variance_)
# transform data
B = pca.transform(A)
print(B)

"""


'\n# Principal Component Analysis\nfrom numpy import array\nfrom sklearn.decomposition import PCA\n# define a matrix\nA = array([[1, 2,7], [3, 4,8], [5, 6,9]])\nprint(A)\n# create the PCA instance\npca = PCA(2)\n# fit on data\npca.fit(A)\n# access values and vectors\nprint(pca.components_)\nprint(pca.explained_variance_)\n# transform data\nB = pca.transform(A)\nprint(B)\n\n'